In [1]:
import os
import re
import pandas as pd

In [2]:
# load and have a look at our data
df = pd.read_csv(f'{os.getcwd()}/poems.csv')
df.head()

,Unnamed: 0,title_of_poem,0
0,0,vtorojfutbol,"Рассеян утренник тяжелый, На босу ногу день пр..."
1,1,natemnomnebe,"На темном небе, как узор, Деревья траурные выш..."
2,2,ne-govori-nikomu,"Не говори никому,Всё, что ты видел, забудь –Пт..."
3,3,ewyo-my-zhizn-yu-polny-v-vysshej-mere,"Ещё мы жизнью полны в высшей мере,Ещё гуляют в..."
4,4,devpolunochnykhotvaga,...Дев полуночных отвага И безумных звезд разб...


In [3]:
# deleting unnecessary columns and rename 
def first_cleaning(df: pd.DataFrame):
    df = df.drop(['Unnamed: 0'], axis = 1)
    df = df.rename(columns={'0': 'text_of_poem'})
    return df

df = first_cleaning(df)
df.head()

,title_of_poem,text_of_poem
0,vtorojfutbol,"Рассеян утренник тяжелый, На босу ногу день пр..."
1,natemnomnebe,"На темном небе, как узор, Деревья траурные выш..."
2,ne-govori-nikomu,"Не говори никому,Всё, что ты видел, забудь –Пт..."
3,ewyo-my-zhizn-yu-polny-v-vysshej-mere,"Ещё мы жизнью полны в высшей мере,Ещё гуляют в..."
4,devpolunochnykhotvaga,...Дев полуночных отвага И безумных звезд разб...


## Main cleaning

In [4]:
def basic_text_preparation(text_prep: list) -> list:
    result = []
    for i in text_prep:
        cleaned_text = re.sub(r'[А-ЯЁ]\.', ' ', i) # removes every letter followed by a dot
        cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text) # removes punctuation
        cleaned_text = re.sub("[^а-яА-ЯёЁ]+", " ", cleaned_text) # deletes latin letters
        cleaned_text = re.sub(r'[0-9]', " ", cleaned_text) # deletes numbers
        result.append(cleaned_text)
    
    return list(set(result))

def main_work_with_poem(df: pd.DataFrame) -> list:
    prepared_poems = []
    for i in df['text_of_poem']:
        text_prep = re.findall('[А-Я][^А-Я]*', i)
        # to get rid of footsteps in last phrase I`m compelled to delete it, 
        # which looks like that: some last phrase. May 1234 page 43
        last_string = text_prep[-1][:text_prep[-1].find('.')]
        text_prep.pop()
        text_prep.append(last_string)
        
        text_prep = basic_text_preparation(text_prep=text_prep)
        text_prep = [a.lower() for a in text_prep] # converts it to lower case
        text_prep = [a.rstrip() for a in text_prep] # removes spaces from left and right sides
        text_prep = [a.lstrip() for a in text_prep]
        
        # now we should remove tiny sentences, because they won't have a good influence on our model
        # and we should remove empty strings if there're
        clean_text = []
        for idx, val in enumerate(text_prep): # removing empty strings
            if text_prep[idx] in ['', ' ']:
                pass
            else:
                clean_text.append(val)

        text_prep = []        
        for val in clean_text: # removing string having less than 10 letters
            if len(val) <= 10:
                pass
            else:
                text_prep.append(val)
        
        text_prep_str = '\n'.join(text_prep)
        prepared_poems.append(text_prep_str)
    
    return prepared_poems


In [5]:
# creating one dataframe with prepared poems
prepared_poems = main_work_with_poem(df = df)
prepared_df = pd.DataFrame()
prepared_df['title_of_poem'] = df['title_of_poem']
prepared_df['text'] = prepared_poems
prepared_df['artist'] = 'Осип Мандельштам'
prepared_df = prepared_df.drop('title_of_poem', axis = 1)
prepared_df.head()

,text_of_poem
0,играют мальчики в футбол\nкто мяч толкает угло...
1,о месяц только ты не суживай\nвысоких неживых ...
2,или чернику в лесу\nтолько уста разомкнешь\nил...
3,и пишут звездоносно и хвостато\nещё машинка но...
4,воспаленной головы\nвыстрел пушечный в подвалы...


In [6]:
# saving prepared df into csv file and checking whether it's correct written
prepared_df.to_csv(f'{os.getcwd()}/cleaned_poems_of_mandelshtamp.csv', index=False, encoding='utf-8')
test = pd.read_csv('cleaned_poems_of_mandelshtamp.csv')
test.head()

,text_of_poem
0,играют мальчики в футбол\nкто мяч толкает угло...
1,о месяц только ты не суживай\nвысоких неживых ...
2,или чернику в лесу\nтолько уста разомкнешь\nил...
3,и пишут звездоносно и хвостато\nещё машинка но...
4,воспаленной головы\nвыстрел пушечный в подвалы...
